# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [ ]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [ ]:
def get_book_data(book_url: str) -> dict:
    """
    Извлекает данные о книге с указанной страницы сайта Books to Scrape.
    
    Args:
        book_url (str): URL страницы книги для парсинга.
        
    Returns:
        dict: Словарь с данными о книге, включающий:
            - title: название книги
            - price: цена
            - rating: рейтинг (количество звезд)
            - availability: количество в наличии
            - description: описание книги
            - product_info: словарь с дополнительной информацией из таблицы
              Product Information (UPC, Product Type, Price (excl. tax) и т.д.)
    """
    response = requests.get(book_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Название книги
    title = soup.find('h1').text.strip()
    
    # Цена
    price = soup.find('p', class_='price_color').text.strip()
    
    # Рейтинг (количество звезд)
    rating_classes = soup.find('p', class_='star-rating')['class']
    rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    rating = 0
    for class_name in rating_classes:
        if class_name in rating_map:
            rating = rating_map[class_name]
            break
    
    # Количество в наличии
    availability = soup.find('p', class_='instock availability').text.strip()
    
    # Описание
    product_description = soup.find('div', id='product_description')
    description = ''
    if product_description:
        description_tag = product_description.find_next_sibling('p')
        if description_tag:
            description = description_tag.text.strip()
    
    # Дополнительная информация из таблицы Product Information
    product_info = {}
    table = soup.find('table', class_='table table-striped')
    if table:
        rows = table.find_all('tr')
        for row in rows:
            th = row.find('th')
            td = row.find('td')
            if th and td:
                key = th.text.strip()
                value = td.text.strip()
                product_info[key] = value
    
    return {
        'title': title,
        'price': price,
        'rating': rating,
        'availability': availability,
        'description': description,
        'product_info': product_info,
        'url': book_url
    }

In [ ]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [ ]:
def scrape_books(is_save: bool = False) -> list:
    """
    Собирает данные обо всех книгах со всех страниц каталога Books to Scrape.
    
    Проходит по всем страницам каталога (page-1.html, page-2.html и т.д.),
    находит ссылки на книги и извлекает данные о каждой книге.
    
    Args:
        is_save (bool): Если True, сохраняет результаты в файл books_data.txt.
                        По умолчанию False.
        
    Returns:
        list: Список словарей с данными о всех книгах.
    """
    base_url = 'http://books.toscrape.com/catalogue/'
    all_books = []
    page = 1
    
    while True:
        if page == 1:
            page_url = f'{base_url}index.html'
        else:
            page_url = f'{base_url}page-{page}.html'
        
        try:
            response = requests.get(page_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Находим все ссылки на книги на странице
            books_on_page = soup.find_all('article', class_='product_pod')
            
            if not books_on_page:
                break
            
            for book_article in books_on_page:
                # Находим ссылку на страницу книги
                link_tag = book_article.find('h3').find('a')
                if link_tag:
                    book_relative_url = link_tag['href']
                    # Убираем относительный путь, если он есть
                    if book_relative_url.startswith('../../../'):
                        book_relative_url = book_relative_url.replace('../../../', '')
                    elif book_relative_url.startswith('../../'):
                        book_relative_url = book_relative_url.replace('../../', '')
                    elif book_relative_url.startswith('../'):
                        book_relative_url = book_relative_url.replace('../', '')
                    
                    book_full_url = f'{base_url}{book_relative_url}'
                    
                    try:
                        book_data = get_book_data(book_full_url)
                        all_books.append(book_data)
                        print(f"Собрана книга: {book_data['title']}")
                    except Exception as e:
                        print(f"Ошибка при парсинге книги {book_full_url}: {e}")
                        continue
            
            page += 1
            time.sleep(0.5)  # Небольшая задержка, чтобы не перегружать сервер
            
        except requests.exceptions.HTTPError:
            # Если страница не найдена, значит мы дошли до конца
            break
        except Exception as e:
            print(f"Ошибка при обработке страницы {page}: {e}")
            break
    
    if is_save:
        with open('books_data.txt', 'w', encoding='utf-8') as f:
            for book in all_books:
                f.write(f"Название: {book['title']}\n")
                f.write(f"Цена: {book['price']}\n")
                f.write(f"Рейтинг: {book['rating']} звезд\n")
                f.write(f"В наличии: {book['availability']}\n")
                f.write(f"Описание: {book['description']}\n")
                f.write(f"URL: {book['url']}\n")
                f.write("Дополнительная информация:\n")
                for key, value in book['product_info'].items():
                    f.write(f"  {key}: {value}\n")
                f.write("\n" + "="*80 + "\n\n")
        print(f"\nДанные сохранены в файл books_data.txt")
    
    return all_books

In [ ]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
def run_scheduled_scraping():
    """
    Запускает функцию сбора данных с сохранением в файл.
    """
    print(f"Запуск сбора данных в {time.strftime('%Y-%m-%d %H:%M:%S')}")
    scrape_books(is_save=True)
    print(f"Сбор данных завершен в {time.strftime('%Y-%m-%d %H:%M:%S')}")


# Настройка расписания: каждый день в 19:00
schedule.every().day.at("19:00").do(run_scheduled_scraping)

print("Расписание настроено: сбор данных будет запускаться каждый день в 19:00")
print("Для тестирования можно изменить время на ближайшее (например, '19:05')")

# Бесконечный цикл для проверки расписания
# Для тестирования можно использовать ближайшее время
# schedule.every().day.at("19:05").do(run_scheduled_scraping)

while True:
    schedule.run_pending()
    time.sleep(60)  # Проверяем расписание каждую минуту

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py -v

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```